# Optical Selection Validation Plots

Plots:
- Number of candidates before and after requirements, signal and background sample, also data/mc
- Rectangular cuts, data mc
- Difference between the scores

In [ ]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt

from IPython.display import clear_output

%matplotlib inline

In [ ]:
pd.options.display.max_columns = 999
gr      = 1.618
flux_corr = 1.028

mass_p= 0.93827 #GeV
mass_e= 0.00511 #GeV
# Minimum reconstructable energies:
min_e = 0.02+mass_e # 20MeV
min_p = 0.04+mass_p # 40MeV
min_pi = 0.17

# dead wires
z_dead_start = 675
z_dead_end=z_dead_start+100

outdir = "../../Output/flash_validation/"

In [ ]:
def true_thresholds_1e1p(row):
    isProton = False
    isPion = False
    if row["true_1eX_signal"]:
        for pdg,E in zip(*row[["nu_daughters_pdg","nu_daughters_E"]]):
            if pdg==2212 and E>min_p:
                isProton = True
            if pdg in [-211,211,111] and E>min_pi:
                isPion = True
    return isProton and not isPion

def maxflash(row,start):
    nr_fl = len(row["flash_time"])
    if nr_fl == 0:
        return 0
    elif nr_fl == 1 and start < row["flash_time"][0] < start+1.6:
        return row["flash_PE"][0]
    elif nr_fl == 1:
        return 0
    else:
        pe_arr = []
        for time,pe in zip(*row[["flash_time","flash_PE"]]):
            if start < time < (start+1.6):
                pe_arr.append(pe)
        if len(pe_arr)==0:
            return 0
        else:
            return max(pe_arr)

In [ ]:
gpvmsdir = '/run/user/1002/gvfs/sftp:host=uboonegpvm02.fnal.gov,user=wvdp'
inputlist = []
inputlist += glob.glob("/home/wouter/Templates/May/data_bnb_a_0s0t/*.root")
inputlist += glob.glob("/home/wouter/Templates/May/data_bnb_b_0s0t/*.root")
filelist  = []
for fname in inputlist:
    filelist.append(fname)
print(len(filelist))

sample = "bnb"
start_time = 3.3

In [ ]:
# Main datafram
columns= [ # True
          "run", "subrun", "event",
          "bnbweight",
          "true_1eX_signal",
          "nu_daughters_pdg","nu_daughters_E",
          "true_vx","true_vy","true_vz",
           # Reco
          "category",
          "n_primaries", 
          "flash_time", "flash_PE",
          "flash_x","flash_score","TPC_x",
          "flash_passed",
          "chargecenter_candidates_total",
          "number_primary_tracks", "number_primary_showers",
          "shower_passed","track_passed",
]

In [ ]:
# Candidates dataframe
columns_candidates_flat = ["bnbweight","true_1eX_signal","true_vz","n_primaries", 
                           "flash_y", "flash_sy", "flash_z", "flash_sz", "flash_PE_max"
                          ]

columns_candidates = ["number_tracks", "number_showers",
                      "number_primary_tracks", "number_primary_showers",
                      "shower_passed","track_passed",
                      "flash_passed",
                      "chargecenter_candidates_x",
                      "chargecenter_candidates_y",
                      "chargecenter_candidates_z",
                      "chargecenter_candidates_total"
                     ]

In [ ]:
if 0:
    from root_pandas import read_root
    chunks = []
    chunks_candidate = []
    
    for i_f,fname in enumerate(filelist):
        clear_output(wait=True)
        print (i_f)
        dftemp=read_root(fname,"wouterNueCC/pandoratree",columns=columns)  
        dftemp_candidate = read_root(fname, 'wouterNueCC/pandoratree',
                                     columns=columns_candidates + columns_candidates_flat,
                                     flatten=columns_candidates)

        chunks.append(dftemp)
        chunks_candidate.append(dftemp_candidate)
    
    clear_output(wait=True)
    print ("Done loading, start processing df")
        
    df = pd.concat(chunks,ignore_index=True,copy=False) 
    df["true_z_deadwire"] = ~df['true_vz'].between(z_dead_start,z_dead_end)
    #df["true_1e1p_signal"] = df.apply(true_thresholds_1e1p,axis=1)
    df.drop(["nu_daughters_pdg","nu_daughters_E"],inplace=True,axis=1)
    df["bnbweight"]*=flux_corr
    df["passed_primaries"] = df["flash_passed"].apply(lambda x: sum(x>-2))
    df["flash_PE_max"] = df.apply(lambda x: maxflash(x,start_time), axis=1)
    
    df.to_pickle("df_"+sample+"_flash.pckl")
    
    
    print ("Done loading, start processing df_candidate")
    df_candidate = pd.concat(chunks_candidate,ignore_index=True,copy=False) 
    df_candidate["true_z_deadwire"] = ~df_candidate['true_vz'].between(z_dead_start,z_dead_end)
    # Candidate is good if:
    df_candidate.eval('good = (true_1eX_signal==1) & (number_primary_tracks==track_passed) \
                       & (number_primary_showers==shower_passed) & (true_z_deadwire==1) \
                       & (chargecenter_candidates_total>1)', inplace=True)
    
    df_candidate.eval('bad = (track_passed==0) & (shower_passed==0) & (chargecenter_candidates_total>1)', inplace=True)
    df_candidate["bnbweight"]*=flux_corr
    df_candidate.eval('diff_z = chargecenter_candidates_z - flash_z', inplace=True)
    df_candidate.eval('diff_sz = diff_z/flash_sz', inplace=True)
    df_candidate.eval('diff_y = chargecenter_candidates_y - flash_y', inplace=True)
    df_candidate.eval('diff_sy = diff_y/flash_sy', inplace=True)
    df_candidate.eval('charge_ratio = flash_PE_max/chargecenter_candidates_total', inplace=True)
    df_candidate.to_pickle("df_"+sample+"_flash_candidate.pckl")

    print("Done processing")
else:
    spill = 1.6
    nue_pot = 2.42e22
    df_nue = pd.read_pickle("df_nue_flash.pckl")
    df_nue_candidate = pd.read_pickle("df_nue_flash_candidate.pckl")
    print("nue loaded")
    
    nu_events = 177100
    nu_pot = 1.79e20
    df_nu = pd.read_pickle("df_nu_flash.pckl")
    df_nu_candidate = pd.read_pickle("df_nu_flash_candidate.pckl")
    print("nu loaded")
    
    ext_scale = 1./3.364
    df_ext = pd.read_pickle("df_bnbext_flash.pckl")
    df_ext_candidate = pd.read_pickle("df_bnbext_flash_candidate.pckl")
    print("ext loaded")
    
    bnb_pot = 4.367e19
    df_bnb = pd.read_pickle("df_bnb_flash.pckl")
    df_bnb_candidate = pd.read_pickle("df_bnb_flash_candidate.pckl")
    print("bnb loaded")
    
    bnb_pot = 4.367e19
    df_intime = pd.read_pickle("df_intime_flash.pckl")
    df_intime_candidate = pd.read_pickle("df_intime_flash_candidate.pckl")
    print("intime loaded")

In [ ]:
#Data
nu_scale = bnb_pot/nu_pot
nue_scale = bnb_pot/nue_pot
df_ext["bnbweight"]=ext_scale
df_bnb["bnbweight"]=1

df_nue[["flash_PE","flash_time"]].head(10)

nu_flash = np.hstack(df_nu["flash_time"])
bnb_flash = np.hstack(df_bnb["flash_time"])
ext_flash = np.hstack(df_ext["flash_time"])
nu_flash_weight = np.hstack(df_nu["bnbweight"])


fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(7*gr,4),sharex=False)
x_min = -0.0
x_max = 8.8
N=44
bhalf = (x_max-x_min)/(2*N)
x_ticks=range(N) 

offset = 0.04
bins,edges = np.histogram(bnb_flash-offset,bins=N,range=(x_min,x_max))
ax[0].errorbar(edges[:-1]+bhalf, bins, xerr=bhalf, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[0].hist(ext_flash-offset-0.406,alpha=1.0,bins=N, range = (x_min, x_max), weights = [ext_scale]*len(ext_flash), label = "BNBext" )
ax[0].hist(nu_flash,alpha=1.0,bins=N, range = (x_min, x_max), weights = [nu_scale]*len(nu_flash), label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )

ax[0].axvspan(x_min,3.2, alpha=0.1, color="red")
ax[0].axvspan(3.2,4.8, alpha=0.1, color="green",label="1.6$\: \mu$s window" )
ax[0].axvspan(4.8,x_max, alpha=0.1, color="red")

ax[0].set_title(r"Time (all flashes)")
ax[0].set_xlabel(r"flash time wrt beam trigger [$\mu$s]")
ax[0].set_ylabel(r"Events per bin")
ax[0].legend(loc="upper right")
ax[0].set_xlim(x_min,x_max)

x_min = 10
x_max = 30000
N=32
bhalf = (x_max-x_min)/(2*N)
x_ticks=range(N) 
edges=np.logspace(np.log10(x_min),np.log10(x_max), N)
x_err= [(edges[i+1]-edges[i])/2 for i in range(N-1)]

edges_mid = [ edges[i]+(edges[i+1]-edges[i])/2 for i in range(N-1)]
                         
bins,_ = np.histogram(df_bnb["flash_PE_max"],bins=edges)
ax[1].errorbar(edges_mid, bins, xerr=x_err, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[1].hist(df_ext["flash_PE_max"],alpha=1.0,bins=edges, weights = df_ext["bnbweight"], label = "BNBext" )
ax[1].hist(df_nu["flash_PE_max"],alpha=1.0,bins=edges, weights = df_nu["bnbweight"]*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )
ax[1].set_title(r"Photo-electrons (brightest flash in window)")
ax[1].set_xlabel(r"# PE in flash")
ax[1].set_xlim(x_min,x_max)
ax[1].axvspan(x_min,50, alpha=0.1, color="red")
ax[1].axvspan(50,x_max, alpha=0.1, color="green",label=">$50 \:$PE cut" )
ax[1].legend(loc="upper left")
ax[1].set_xscale("log", nonposx='clip')

fig.tight_layout()
plt.savefig( outdir+"time_pe.pdf")

print("\nEvents have no flash in window:")
print("BNB\t\t", sum(df_bnb.query("flash_PE_max<0.1")["bnbweight"]))
print("MC + BNBext\t", round(sum(df_nu.query("flash_PE_max<0.1")["bnbweight"]*nu_scale)+sum(df_ext.query("flash_PE_max<0.1")["bnbweight"])))
print("MCC signal (nu_e 1eX)\t", round(10*sum(df_nue.query("true_1eX_signal==1 & flash_PE_max<0.1")["bnbweight"]*nue_scale))/10)

print("Events have a flash in window:")
print("BNB\t\t", round(sum(df_bnb.query("flash_PE_max>0.1")["bnbweight"])))
print("MC + BNBext\t", round(sum(df_nu.query("flash_PE_max>0.1")["bnbweight"]*nu_scale)+sum(df_ext.query("flash_PE_max>0.1")["bnbweight"])))
print("MCC signal (nu_e 1eX)\t", round(sum(df_nue.query("true_1eX_signal==1 & flash_PE_max>0.1")["bnbweight"]*nue_scale)))

print("Events have a 50PE flash in window:")
print("BNB\t\t", round(sum(df_bnb.query("flash_PE_max>50")["bnbweight"])))
print("MC + BNBext\t", round(sum(df_nu.query("flash_PE_max>50")["bnbweight"]*nu_scale)+sum(df_ext.query("flash_PE_max>50")["bnbweight"])))
print("MCC signal (nu_e 1eX)\t", round(sum(df_nue.query("true_1eX_signal==1 & flash_PE_max>50")["bnbweight"]*nue_scale)))


In [ ]:
df_nue[["flash_time","flash_PE"]].head(100)

events_intime = 115323
intime_scaling = 10.279*(nu_events/events_intime)*(bnb_pot/nu_pot)

df_intime["bnbweight"]=intime_scaling

In [ ]:
#fig,ax=plt.subplots(ncols=3,nrows=1,figsize=(5*gr,3),sharex=True,sharey=False)

plt.figure(figsize=(6*gr,6))
ax=np.empty(5)
ax0 = plt.subplot2grid((4, 3), (1, 0), rowspan=2)
ax1 = plt.subplot2grid((4, 3), (0, 1), rowspan=2)
ax2 = plt.subplot2grid((4, 3), (0, 2), rowspan=2)
ax3 = plt.subplot2grid((4, 3), (2, 1), rowspan=2)
ax4 = plt.subplot2grid((4, 3), (2, 2), rowspan=2)


x_min = -0.5
x_max = 8.5
N=9
x_ticks=range(N) 

str_query = "flash_PE_max>50"
str_query_signal = "true_1eX_signal==1 & flash_PE_max>50"

ax0.hist(df_nue.query(str_query_signal)["n_primaries"],alpha=0.4,bins=N, range = (x_min, x_max), rwidth=0.75, label = "Before cuts", weights= df_nue.query(str_query_signal)["bnbweight"]*nue_scale, color = "k"  )
ax0.hist(df_nue.query(str_query_signal)["passed_primaries"],alpha=0.8,bins=N, color="C2", range = (x_min, x_max), rwidth=0.75, label = "After cuts" , weights = df_nue.query(str_query_signal)["bnbweight"]*nue_scale )
ax0.set_xticks(x_ticks)
ax0.set_title(r"MC signal ($\nu_e$ 1eX)")
ax0.set_xlabel(r"# $\nu$ candidates")
ax0.set_ylabel(r"# Events per bin")
ax0.legend()

#Data
bins,edges = np.histogram(df_bnb.query(str_query)["n_primaries"],bins=N,range=(x_min,x_max))
ax1.errorbar(edges[:-1]+0.5, bins, xerr=0.5*0.75, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax1.hist(df_ext.query(str_query)["n_primaries"],alpha=1.0,bins=N, range = (x_min, x_max), rwidth=0.75, weights = df_ext.query(str_query)["bnbweight"], label = "BNBext" )
ax1.hist(df_nu.query(str_query)["n_primaries"],alpha=1.0,bins=N, range = (x_min, x_max), rwidth=0.75, weights = df_nu.query(str_query)["bnbweight"]*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )
ax1.set_title(r"BNB (Before cuts)")
ax1.set_xlabel(r"# $\nu$ candidates")
ax1.legend(loc="upper center")
ax1.set_ylim(0,78000)

bins,edges = np.histogram(df_bnb.query(str_query)["passed_primaries"],bins=N,range=(x_min,x_max))
ax2.errorbar(edges[:-1]+0.5, bins, xerr=0.5*0.75, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax2.hist(df_ext.query(str_query)["passed_primaries"],alpha=1.0,bins=N, range = (x_min, x_max), rwidth=0.75, weights = df_ext.query(str_query)["bnbweight"], label = "BNBext" )
ax2.hist(df_nu.query(str_query)["passed_primaries"],alpha=1.0,bins=N, range = (x_min, x_max), rwidth=0.75, weights = df_nu.query(str_query)["bnbweight"]*nu_scale, label=r"BNB $\nu_\mu$",  bottom = lower )
ax2.set_title(r"BNB (After cuts)")
ax2.set_xlabel(r"# $\nu$ candidates")
ax2.set_ylim(0,78000)

bins,edges = np.histogram(df_ext.query(str_query)["n_primaries"],weights=df_ext.query(str_query)["bnbweight"] ,bins=N,range=(x_min,x_max))
ax3.errorbar(edges[:-1]+0.5, bins, xerr=0.5*0.75, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB ext (4.4e19 POT)")
ax3.hist(df_intime.query(str_query)["n_primaries"],alpha=1.0,bins=N, range = (x_min, x_max), color="C4", rwidth=0.75, weights = df_intime.query(str_query)["bnbweight"], label=r"CORSIKA Cosmic intime")
ax3.set_title(r"BNB ext (Before cuts)")
ax3.set_xlabel(r"# $\nu$ candidates")
ax3.legend(loc="upper center")
ax3.set_ylim(0,78000)


bins,edges = np.histogram(df_ext.query(str_query)["passed_primaries"],weights=df_ext.query(str_query)["bnbweight"] ,bins=N,range=(x_min,x_max))
ax4.errorbar(edges[:-1]+0.5, bins, xerr=0.5*0.75, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB ext (4.4e19 POT)")
ax4.hist(df_intime.query(str_query)["passed_primaries"],alpha=1.0,bins=N, color="C4", range = (x_min, x_max), rwidth=0.75, weights = df_intime.query(str_query)["bnbweight"], label=r"Cosmic intime")
ax4.set_title(r"BNB ext (After cuts)")
ax4.set_xlabel(r"# $\nu$ candidates")
ax4.set_ylim(0,78000)

plt.tight_layout()
plt.savefig( outdir+"n_candidates.pdf")

In [ ]:
df_ext_candidate["bnbweight"]=ext_scale

In [ ]:
x_axis_arr = ["diff_z", "diff_sz", "diff_y", "diff_sy","charge_ratio"]
x_label_arr = [r"Charge $z$ - Flash $z$ [cm]", "(Charge $z$ - Flash $z$) / Flashwidth $z$",
               r"Charge $y$ - Flash $y$ [cm]", "(Charge $y$ - Flash $y$) / Flashwidth $y$",
               r"Charge light ration [PE/ (200$e^-$)]"]

x_min_arr =  [-200,-1.8,-120,-4.0,0]
x_max_arr =  [150,1.4,120,4.0,20]
bins_arr = [30,30,30,30,30]
y_max_arr = [0.033,4.0,0.013,0.55,0.025]
cut_arr = [105,1.0,95,2.1,3.0]

for x_axis, x_label, x_min, x_max, bins, y_max, cut in zip(x_axis_arr, x_label_arr, x_min_arr, x_max_arr, bins_arr, y_max_arr, cut_arr):
    
    fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(6*gr,4.5),sharex=True)
    ax[0].hist(df_nue_candidate.query("good==1")[x_axis], range=(x_min,x_max),bins=bins,histtype="step", 
               stacked=False, fill=False, weights=df_nue_candidate.query("good==1")["bnbweight"], label = r"Signal matched ($\nu_e$ 1eX)", density = 1,lw=1.5)
    ax[0].hist(df_nue_candidate.query("bad==1")[x_axis] , range=(x_min,x_max),bins=bins,histtype="step", 
               stacked=False, fill=False, weights=df_nue_candidate.query("bad==1")["bnbweight"], label = r"Background ($\nu_e$)", density = 1,lw=1.5)
    
    ax[0].hist(df_ext_candidate.query("bad==1")[x_axis] , range=(x_min,x_max),bins=bins,histtype="step", 
               stacked=False, fill=False, weights=df_ext_candidate.query("bad==1")["bnbweight"], label = r"Background (BNBext)", density = 1,lw=1.5)
    
    ax[0].axvspan(x_min,-cut, alpha=0.1, color="red")
    ax[0].axvspan(-cut,cut, alpha=0.1, color="green",label="Cut value: "+str(cut) )
    ax[0].axvspan(cut,x_max, alpha=0.1, color="red")
    ax[0].set_title(r"MC (area normalised)")
    ax[0].legend(loc="upper left")
    ax[0].set_xlabel(x_label)
    ax[0].set_ylabel("Area normalised")
    ax[0].set_ylim(0,y_max)
    ax[0].set_xlim(x_min,x_max)
    
    str_query = "flash_passed>-3"
    bhalf = (x_max-x_min)/(2*bins)
    
    values,edges = np.histogram(df_bnb_candidate.query(str_query)[x_axis],bins=bins,range=(x_min,x_max))
    ax[1].errorbar(edges[:-1]+bhalf, values, xerr=bhalf, yerr=np.sqrt(bins), color= "k",fmt=".",label="BNB (4.4e19 POT)")
    lower,_,_ = ax[1].hist(df_ext_candidate.query(str_query)[x_axis],bins=bins, range = (x_min, x_max), weights = df_ext_candidate.query(str_query)["bnbweight"], label = "BNBext" )
    ax[1].hist(df_nu_candidate.query(str_query)[x_axis],alpha=1.0,bins=bins, range = (x_min, x_max), weights = df_nu_candidate.query(str_query)["bnbweight"]*nu_scale, label=r"BNB $\nu_\mu$",  bottom = lower )
    ax[1].set_title(r"Data/MC")
    ax[1].set_xlabel(x_label)
    ax[1].legend(loc="upper left")
    
    ax[1].set_ylabel("Events per bin")
    ax[1].axvspan(x_min,-cut, alpha=0.1, color="red")
    ax[1].axvspan(-cut,cut, alpha=0.1, color="green")
    ax[1].axvspan(cut,x_max, alpha=0.1, color="red")
    #ax[1].set_ylim(0,78000)
    
    print(len(df_bnb_candidate.query(str_query)[x_axis]))
    fig.tight_layout()
    fig.savefig( outdir+x_axis+".pdf")

In [ ]:
# make plots of the flahsmatching! (slow)

diff_score_arr = []
diff_x_arr = []
score_max_arr = []
weight_arr = []
weight2_arr = []
light_charge_ratio_arr = []

light_charge_ratio_good = []
light_charge_ratio_bad = []
diff_x_good = []
diff_x_bad = []
weights_good = []
weights_bad = []

labels = ["nu","nue","bnb","ext"]
df_arr = [df_nu,df_nue, df_bnb, df_ext]

for df,lab in zip(df_arr,labels):
    diff_score = []
    diff_x = []
    score_max = []
    weight = []
    weight2 = []
    light_charge_ratio = []
    
    print(lab)
    for index, row in df.iterrows():
        if len(row["flash_score"])>0:
            if len(row["flash_score"])==1:
                print("1 score?")
                continue;
            diff_score.append( row["flash_score"][0]/row["flash_score"][1] )
            score_max.append( row["flash_score"][0])
            diff_x.append( row["TPC_x"][0]-row["flash_x"][0] )
            weight.append(row["bnbweight"])
            
        if np.any(row["flash_passed"]>-3):
            light_charge_ratio.extend(row["chargecenter_candidates_total"]/row["flash_PE_max"])
            weight2.extend([row["bnbweight"]]*int(row["n_primaries"]))
            
            if lab=="nue":
                for tr_pass, sh_pass, pr_tr, pr_sh, ch_total in zip(*row[["track_passed","shower_passed","number_primary_tracks","number_primary_showers","chargecenter_candidates_total"]]):
                    if row["true_1eX_signal"]==1 and row["true_z_deadwire"]==1 and tr_pass==pr_tr and sh_pass==pr_sh and ch_total>1:
                        #Good
                        light_charge_ratio_good.append(ch_total/row["flash_PE_max"])
                        diff_x_good.append( row["TPC_x"][0]-row["flash_x"][0] )
                        weights_good.append(row["bnbweight"])
                        if len(row["flash_score"])>1:
                            
                    elif tr_pass==0 and sh_pass==0 and ch_total>1:
                        #bad
                        light_charge_ratio_bad.append(ch_total/row["flash_PE_max"])
                        weights_bad.append(row["bnbweight"])
                        if len(row["flash_score"])>1:
                            diff_x_bad.append( row["TPC_x"][0]-row["flash_x"][0] )
            
    diff_score_arr.append(diff_score)
    diff_x_arr.append(diff_x)
    score_max_arr.append(score_max)
    weight_arr.append(weight)
    light_charge_ratio_arr.append(light_charge_ratio)
    weight2_arr.append(weight2)

In [ ]:
print(np.mean(light_charge_ratio_arr[3]))
print(np.mean(light_charge_ratio_good))
print(np.mean(light_charge_ratio_bad))

light_charge_ratio_arr[3]
print(sum(np.array(light_charge_ratio_good)>3)/len(np.array(light_charge_ratio_good)))
print(sum(np.array(light_charge_ratio_arr[3])>3)/len(np.array(light_charge_ratio_arr[3])))

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(6*gr,4.5),sharex=False)

x_min = 0.1
x_max = 3000
N=30
bhalf = (x_max-x_min)/(2*N)
edges=np.logspace(np.log10(x_min),np.log10(x_max), N)
cut = 3.0
y_max = 0.19

x_err= [(edges[i+1]-edges[i])/2 for i in range(N-1)]
edges_mid = [ edges[i]+(edges[i+1]-edges[i])/2 for i in range(N-1)]

ax[0].set_xscale("log", nonposx='clip')
ax[0].hist(light_charge_ratio_good,bins=edges,histtype="step", 
           stacked=False, fill=False, weights=np.array(weights_good)/sum(weights_good), label = r"Signal matched ($\nu_e$ 1eX)", lw=1.5)
ax[0].hist(light_charge_ratio_bad , bins=edges,histtype="step", 
           stacked=False, fill=False, weights=np.array(weights_bad)/sum(weights_bad), label = r"Background ($\nu_e$)", lw=1.5)

ax[0].hist(light_charge_ratio_arr[3],bins=edges,histtype="step", 
           stacked=False, fill=False, weights=np.array(weight2_arr[3])/sum(weight2_arr[3]), label = r"Background (BNBext)", lw=1.5)

ax[0].axvspan(x_min,cut, alpha=0.1, color="red")
ax[0].axvspan(cut,x_max, alpha=0.1, color="green",label="Cut value: 3.0" )
ax[0].set_title(r"MC (area normalised)")
ax[0].legend(loc="upper left")
ax[0].set_xlabel(r"Charge / Light ratio [sum hit integrals / PE]")
ax[0].set_ylabel("Area normalised")
ax[0].set_ylim(0,y_max)
ax[0].set_xlim(x_min,x_max)

ax[1].set_xscale("log", nonposx='clip')
bins,_ =  np.histogram(np.array(light_charge_ratio_arr[2]),bins=edges)
ax[1].errorbar(edges_mid, bins, xerr=x_err, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[1].hist(light_charge_ratio_arr[3],alpha=1.0,bins=edges, weights = np.ones(len(light_charge_ratio_arr[3]))*ext_scale, label = "BNBext" )
ax[1].hist(light_charge_ratio_arr[0],alpha=1.0,bins=edges, weights = np.array(weight2_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )

ax[1].axvspan(x_min,cut, alpha=0.1, color="red")
ax[1].axvspan(cut,x_max, alpha=0.1, color="green",label="Cut value: 3.0" )
ax[1].legend(loc="upper left")
ax[1].set_ylabel("Events per bin")
ax[1].set_xlabel(r"Charge / Light ratio [sum hit integrals / PE]")
ax[1].set_xlim(x_min,x_max)
ax[1].set_ylim(0,27000)

fig.tight_layout()
plt.savefig( outdir+"charge_ratio.pdf")

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(6*gr,4.5),sharex=False)
x_min = -100
x_max = 100
N=20
bhalf = (x_max-x_min)/(2*N)
x_ticks=range(N) 


#bins,edges = np.histogram(score_max_arr[2],bins=N, range=(x_min,x_max))
#ax[0].errorbar(edges[:-1]+bhalf, bins, xerr=bhalf, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
#lower,_,_ = ax[0].hist(score_max_arr[3],alpha=1.0,bins=N, range=(x_min,x_max), weights = np.ones(len(score_max_arr[3]))*ext_scale, label = "BNBext" )
#ax[0].hist(score_max_arr[0],alpha=1.0,bins=edges, weights = np.array(weight_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )
#ax[0].set_xlim(x_min,x_max)
#ax[0].set_ylim(0.1500)
#ax[0].set_ylabel("Events per bin")
#ax[0].set_xlabel("Highest flash-match score")
#ax[0].legend(loc="upper right")

x_min = -200
x_max = 200
N=60
bhalf = (x_max-x_min)/(2*N)
x_ticks=range(N) 

bins,edges = np.histogram(diff_x_arr[2],bins=N, range=(x_min,x_max))
ax[1].errorbar(edges[:-1]+bhalf, bins, xerr=bhalf, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[1].hist(diff_x_arr[3],alpha=1.0,bins=N, range=(x_min,x_max), weights = np.ones(len(diff_score_arr[3]))*ext_scale, label = "BNBext" )
ax[1].hist(diff_x_arr[0],alpha=1.0,bins=edges, weights = np.array(weight_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )

ax[1].set_xlim(x_min,x_max)
ax[1].set_ylim(0.1500)
ax[1].set_ylabel("Events per bin")
ax[1].set_xlabel("Difference flash x and TPC x [cm]")
ax[1].legend(loc="upper right")

fig.tight_layout()

print(len(diff_score_arr[2]))
plt.savefig( outdir+"flashmatch_x.pdf")

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(6*gr,4.5),sharex=False)
x_min = 10e-6
x_max = 1.5
N=20
bhalf = (x_max-x_min)/(2*N)
x_ticks=range(N) 


bins,edges = np.histogram(score_max_arr[2],bins=N, range=(x_min,x_max))
ax[0].errorbar(edges[:-1]+bhalf, bins, xerr=bhalf, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[0].hist(score_max_arr[3],alpha=1.0,bins=N, range=(x_min,x_max), weights = np.ones(len(score_max_arr[3]))*ext_scale, label = "BNBext" )
ax[0].hist(score_max_arr[0],alpha=1.0,bins=edges, weights = np.array(weight_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )
ax[0].set_xlim(x_min,x_max)
ax[0].set_ylim(0.1500)
ax[0].set_ylabel("Events per bin")
ax[0].set_xlabel("Highest flash-match score")
ax[0].legend(loc="upper right")

x_min = 1
x_max = 1e2
N=20
bhalf = (x_max-x_min)/(2*N)
x_ticks=range(N) 

bins,edges = np.histogram(diff_score_arr[2],bins=N, range=(x_min,x_max))
ax[1].errorbar(edges[:-1]+bhalf, bins, xerr=bhalf, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[1].hist(diff_score_arr[3],alpha=1.0,bins=N, range=(x_min,x_max), weights = np.ones(len(diff_score_arr[3]))*ext_scale, label = "BNBext" )
ax[1].hist(diff_score_arr[0],alpha=1.0,bins=edges, weights = np.array(weight_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )

ax[1].set_xlim(x_min,x_max)
ax[1].set_ylim(0.1500)
ax[1].set_ylabel("Events per bin")
ax[1].set_xlabel("Ratio of two highest scores")
ax[1].legend(loc="upper right")

fig.tight_layout()

print(len(diff_score_arr[2]))
plt.savefig( outdir+"flashmatch.pdf")

In [ ]:
fig,ax=plt.subplots(ncols=2,nrows=1,figsize=(6*gr,4.5),sharex=False)
x_min = 10e-6
x_max = 5.0
N=20
edges=np.logspace(np.log10(x_min),np.log10(x_max), N)
x_err= [(edges[i+1]-edges[i])/2 for i in range(N-1)]
edges_mid = [ edges[i]+(edges[i+1]-edges[i])/2 for i in range(N-1)]

bins,_ = np.histogram(score_max_arr[2],bins=edges,range=(x_min,x_max))
ax[0].errorbar(edges_mid, bins, xerr=x_err, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[0].hist(score_max_arr[3],alpha=1.0,bins=edges, range=(x_min,x_max), weights = np.ones(len(score_max_arr[3]))*ext_scale, label = "BNBext" )
ax[0].hist(score_max_arr[0],alpha=1.0,bins=edges, weights = np.array(weight_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )

ax[0].set_xscale("log", nonposx='clip')
ax[0].set_xlim(x_min,x_max)
ax[0].set_ylim(0.1500)
ax[0].set_ylabel("Events per bin")
ax[0].set_xlabel("Highest flash-match score")
ax[0].legend(loc="upper left")

x_min = 1
x_max = 1e5
N=20
edges=np.logspace(np.log10(x_min),np.log10(x_max), N)
x_err= [(edges[i+1]-edges[i])/2 for i in range(N-1)]
edges_mid = [ edges[i]+(edges[i+1]-edges[i])/2 for i in range(N-1)]

bins,edges = np.histogram(diff_score_arr[2],bins=edges, range=(x_min,x_max))
ax[1].errorbar(edges_mid, bins, xerr=x_err, yerr=np.sqrt(bins),alpha=1.0, color= "k",fmt=".",label="BNB (4.4e19 POT)")
lower,_,_ = ax[1].hist(diff_score_arr[3],alpha=1.0,bins=edges, range=(x_min,x_max), weights = np.ones(len(diff_score_arr[3]))*ext_scale, label = "BNBext" )
ax[1].hist(diff_score_arr[0],alpha=1.0,bins=edges, weights = np.array(weight_arr[0])*nu_scale, label=r"BNB $\nu_\mu$ (MCC)",  bottom = lower )

ax[1].set_xscale("log", nonposx='clip')
ax[1].set_xlim(x_min,x_max)
ax[1].set_ylim(0.1500)
ax[1].set_ylabel("Events per bin")
ax[1].set_xlabel("Ratio of two highest scores")
ax[1].legend(loc="upper right")

fig.tight_layout()

print(len(diff_score_arr[2]))
plt.savefig( outdir+"flashmatch_log.pdf")